In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [36]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tensorflow.python.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
%matplotlib inline
#sns.set(style=’whitegrid’, palette=’muted’, font_scale=1.5)
#rcParams[‘figure.figsize’] = 14, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [2]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [4]:
import pandas as pd
data = pd.read_csv('bitcoin.csv', date_parser = True)
print(data.tail())

         Date          Open          High           Low         Close  \
5  2022-11-04  20208.769531  21209.560547  20188.019531  21147.230469   
6  2022-11-05  21144.832031  21446.886719  21097.634766  21282.691406   
7  2022-11-06  21285.056641  21345.376953  20920.191406  20926.486328   
8  2022-11-07  20924.621094  21053.246094  20489.972656  20602.816406   
9  2022-11-08  20620.449219  20662.337891  19360.029297  20104.789063   

      Adj Close       Volume  
5  21147.230469  64072727950  
6  21282.691406  37846047609  
7  20926.486328  35082693210  
8  20602.816406  53510852236  
9  20104.789063  76600000512  


In [6]:
df = pd.read_csv('bitcoin.csv')


In [8]:
df = pd.read_csv('bitcoin.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-30,20817.982422,20917.005859,20547.462891,20635.603516,20635.603516,31486345556
1,2022-10-31,20633.695313,20795.320313,20287.458984,20495.773438,20495.773438,45668466815
2,2022-11-01,20494.898438,20647.289063,20359.845703,20485.273438,20485.273438,39819303159
3,2022-11-02,20482.958984,20742.810547,20087.134766,20159.503906,20159.503906,55552169483
4,2022-11-03,20162.689453,20382.095703,20086.240234,20209.988281,20209.988281,43228750179


In [22]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
year_2022 = df.loc[(df['Date'] >= '2022-10-30')
                     & (df['Date'] < '2022-11-08')]
year_2022.drop(year_2022[['High','Low','Open','Close']],axis=1)
months_2022 = year_2022.groupby(year_2022['Date'].dt.strftime('%B'))[['Open','Close']].mean()
month_order = [ 'October', 'November']
months_2022 = round(months_2022.reindex(month_order, axis=0),2)
months_2022 = months_2022.reset_index()

In [23]:
fig = px.bar(months_2022, x='Date', y=['Open','Close'], barmode='group', title = 'Monthly Average Open & Close Price - Year 2022')
fig.show()


In [11]:
fig = px.line(df, x="Date", y="Close", title='Bitcoin Close price over time')
fig.show()

In [24]:
model_data = df[['Date','Close']]

In [28]:
df = df.rename(columns = {'Date' : 'Date', 'Closing Price (USD)' : 'Close'})
scaler = MinMaxScaler()
close_price = df.Close.values.reshape(-1, 1)
scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1, 1)

In [33]:
SEQ_LEN = 100
def to_sequences(data, seq_len):
    d = []
    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])
    return np.array(d)
def preprocess(data_raw, seq_len, train_split):
    data = to_sequences(data_raw, seq_len)
    num_train = int(train_split * data.shape[0])
    X_train = data[:num_train:-1]
    y_train = data[:num_train-1]
    X_test = data[num_train:-1]
    y_test = data[num_train:-1]
    return X_train, y_train, X_test, y_test
X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.80)

In [37]:
model = tf.keras.Sequential()
model.add(layers.LSTM(units=32, return_sequences=True,
 input_shape=(99,1), dropout=0.2))
model.add(layers.LSTM(units=32, return_sequences=True,
 dropout=0.2))
model.add(layers.LSTM(units=32, dropout=0.2))
model.add(layers.Dense(units=1))
model.summary()
model.compile(
    loss='mean_squared_error', 
    optimizer='adam'
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 99, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 99, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#data_training = data[data['Date']< '2022-11-04'].copy()
#data_training

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-10-30,20817.982422,20917.005859,20547.462891,20635.603516,20635.603516,31486345556
1,2022-10-31,20633.695313,20795.320313,20287.458984,20495.773438,20495.773438,45668466815
2,2022-11-01,20494.898438,20647.289063,20359.845703,20485.273438,20485.273438,39819303159
3,2022-11-02,20482.958984,20742.810547,20087.134766,20159.503906,20159.503906,55552169483
4,2022-11-03,20162.689453,20382.095703,20086.240234,20209.988281,20209.988281,43228750179
